# Abstract

Interactive plots to visualize South Carolina energy generation and demand seasonality, as well as proportion of in-state energy generation. 

In [ ]:
%store -r energy_SC
%store -r energy_by_BA

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import plotly.express as px

# Daily Energy Generation and Demand

### Assign Colors and Order

In [4]:
custom_color_map = {
    'Demand': 'rgb(228, 26, 28)',  # Red
    'Net Generation': 'rgb(55, 126, 184)',  # Blue
    'Nuclear': 'rgb(77, 175, 74)',   # Green
    'Solar': 'rgb(255, 217, 47)',   # Red
    'Hydro': 'rgb(148, 103, 189)'  # Purple
}

custom_order = ['Demand', 'Net Generation', 'Nuclear', 'Solar', 'Hydro']
custom_colors = [custom_color_map[measurement] for measurement in custom_order]

### Generate Plot

In [6]:
energy_SC_plot = px.line(energy_SC, x="Date", y="MWh", color='Measurement', 
                               title="South Carolina - Daily Energy Generation and Demand",
                               color_discrete_sequence=custom_colors,
                               category_orders={'Measurement': custom_order})
energy_SC_plot.show()

# Clean Energy Generation Proportion

### Create Subset Data Frame

In [8]:
pie_df = energy_SC[energy_SC['Measurement'] != 'Demand']

### Function to Convert Net Generation

In [9]:
def calculate_other_imported(group):
    other_imported = group.loc[group['Measurement'] == 'Net Generation', 'MWh'] - \
                     group.loc[group['Measurement'].isin(['Hydro', 'Solar', 'Nuclear']), 'MWh'].sum()
    
    group.loc[group['Measurement'] == 'Net Generation', 'Measurement'] = 'Other (Imported)'
    group.loc[group['Measurement'] == 'Other (Imported)', 'MWh'] = other_imported
    
    return group

# Apply the function to each group based on the "Date" column
pie_df = pie_df.groupby('Date').apply(calculate_other_imported)

/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_77216/1145063306.py:11: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)



### Assign Colors and Order

In [10]:
custom_color_map_2 = {
    'Nuclear': 'rgb(77, 175, 74)',   # Green
    'Hydro': 'rgb(148, 103, 189)',  # Purple
    'Solar': 'rgb(255, 217, 47)',   # Red
    'Other (Imported)': 'rgb(124,124,124)' #Grey
}

custom_order_2 = ['Nuclear', 'Hydro', 'Solar', 'Other (Imported)']
custom_colors_2 = [custom_color_map_2[measurement] for measurement in custom_order_2]

### Generate Plot

In [12]:
energy_pie = px.pie(pie_df, values='MWh', names='Measurement', 
                               title="South Carolina - Net Electricity Generation Breakdown",
                               color_discrete_sequence=custom_colors_2,
                               category_orders={'Measurement': custom_order_2})

energy_pie.update_layout(
    legend=dict(title='Energy Source'),
    showlegend=True,
    legend_x=0.8,
    title = dict(x=0.5)
)


energy_pie.show()